import needed libraries

In [203]:
import random
import math
import numpy as np

This class is for test case objects that saves different values of each object

In [232]:
class Data:

    # initialize needed values
    def __init__(self, data_address):
        self.address    = data_address      # address of test file
        self.degree     = 0                 # degree of polynomial
        self.low        = 0                 # lower bound of coefs
        self.high       = 0                 # upper bound of coefs
        self.num_points = 0                 # number of points in test case
        self.points     = []                # points of curve

    # read test case
    def load_data(self):
        with open (self.address, 'r') as f:
            self.degree         = int(f.readline())     
            self.low, self.high = map(float, f.readline().split(" "))  
            self.num_points     = int(f.readline())  
            self.points         = [tuple(map(float, f.readline().split(" "))) for _ in range(self.num_points)]  # append the point to list of points  


a class for gene objects, with different methods and values of a gene

In [276]:
class Gene:

    # initialize parameters
    def __init__(self, degree, points, low, high):
        self.gene    = []
        self.degree  = degree
        self.points  = points
        self.low     = low
        self.high    = high
        self.fitness = math.inf


    # create random gene for first population
    def initial_gene(self):
        self.gene = [random.uniform(self.low, self.high) for _ in range(self.degree)]

    # calculate fitness of the gene
    def fitness_function(self):
        y_pred       = [np.polyval(self.gene, point[0]) for point in self.points]  # predicted value for each point
        residuals    = [(y_pred[i] - self.points[i][1])**2 for i in range(len(self.points))]  # SSE for each point
        weight       = 1.0 / np.std(y_pred)  
        weighted_rss = weight * np.sum(residuals)   # calculate weighted RSS

        self.fitness = weighted_rss
        return weighted_rss
    
    # mutate gene
    def mutation(self):

        i = random.randint(0, (self.degree-1)//2)
        j = random.randint((self.degree-1)//2, self.degree-1)
        k = random.randint((self.degree-1)//2, self.degree-1)

        # randomly change 3 coefs in gene
        self.gene[i] = np.round(random.uniform(-1, 1), 6)
        self.gene[j] = np.round(random.uniform(-1, 1), 6)
        self.gene[k] = np.round(random.uniform(-1, 1), 6)
        
        self.fitness = self.fitness_function()


    # define a local search to search for neighbors of a gene that is better than itself
    def local_search(self):

        neighbors = []
        for i in range(self.degree):   # find as many neighbors as degree of polynimal and change one index at a time
            genei = Gene(self.degree, self.points, self.low, self.high)
            rnd = np.round(random.uniform(-1, 1), 6)
            genei.gene = self.gene
            genei.gene[i] = rnd
            genei.fitness = genei.fitness_function()
            neighbors.append((genei.fitness, genei))

        # sort neighbors and select the best one or the child itself
        neighbors = sorted(neighbors, key=lambda x: x[0])
        if neighbors[0][0] < self.fitness:
            return neighbors[0][1]
        else:
            return self



a class for population of genes

In [279]:
class Population:

    # initialize population's parameters
    def __init__(self, data, pop_size=100):
        self.population_size             = pop_size
        self.population                  = []
        self.points                  = data.points
        self.low, self.high, self.degree = data.low, data.high, data.degree

    # generate first population randomly
    def initialize_population(self):
        for _ in range(self.population_size):
            genei = Gene(self.degree, self.points, self.low, self.high)
            genei.initial_gene()
            self.population.append(genei)
    

    # selest parents from population for crossover using tournament selection
    def parent_selection(self, N=3):
        # choose random N candidates to attend tournament
        father_candidates = [random.choice(self.population) for _ in range(N)]
        mother_candidates = [random.choice(self.population) for _ in range(N)]

        # choose the best one of N candidates to be a parent
        father_vals = [(father.fitness, father) for father in father_candidates]
        mother_vals = [(mother.fitness, mother) for mother in mother_candidates]

        # sort candidates
        sorted_fathers = sorted(father_vals, key=lambda x: x[0])
        sorted_mothers = sorted(mother_vals, key=lambda x: x[0])

        # select parents with min fitness
        best_father = sorted_fathers[0][1]
        best_mother = sorted_mothers[0][1]
        
        return best_father, best_mother

    # selest parents from population for crossover using rank-based selection
    def parent_selection_rank_based(self):
        # sort population based on fitness
        fitness_values = [gene.fitness for gene in self.population]
        ranked_indices = np.argsort(fitness_values)
        ranked_population = [self.population[i] for i in ranked_indices]
        
        # rank calculation for selection 
        selection_p = np.arange(1, self.population_size + 1) / np.sum(np.arange(1, self.population_size + 1))
        
        # select parents based on rank and selection probabilities
        selected_parents = np.random.choice(ranked_population, size=2, replace=False, p=selection_p)
        
        return selected_parents[0], selected_parents[1]


    # crossover function (order)
    def crossover(self, father, mother):

        child1l, child2l = [], []
        child1, child2   = Gene(self.degree, self.points, self.low, self.high), Gene(self.degree, self.points, self.low, self.high)

        random_point = random.randint(0, (self.degree-1)//2)    # select a random point and cut 1 of parents for each child from there

        # put first part of 1 parent to each child
        child1l = [father.gene[i] for i in range(random_point)]
        child2l = [mother.gene[i] for i in range(random_point)]

        # put the rest of other parent
        for i in range(random_point, self.degree):
            child1l.append(mother.gene[i])
            child2l.append(father.gene[i])

        # initialize childs gene object
        child1.gene    = child1l
        child2.gene    = child2l
        child1.fitness = child1.fitness_function()
        child2.fitness = child2.fitness_function()
        
        # select the best generated child
        if child1.fitness < child2.fitness:
            return child1
        else:
            return child2
    
    # blend crossover
    def blx_alpha_crossover(self, father, mother, alpha=0.5):
        child = []
        for i in range(self.degree):
            min_val = min(father.gene[i], mother.gene[i])
            max_val = max(father.gene[i], mother.gene[i])
            
            parameter_range = max_val - min_val
            lower_bound     = min_val - alpha * parameter_range
            upper_bound     = max_val + alpha * parameter_range
            
            # check domain
            if lower_bound < self.low:
                lower_bound = self.low
            if upper_bound > self.high:
                upper_bound = self.high

            # generate a random value within the range 
            child_val = np.random.uniform(lower_bound, upper_bound)
            child.append(child_val)
        
        # initialize child object
        child1 = Gene(self.degree, self.points, self.low, self.high)
        child1.gene = child
        child1.fitness = child1.fitness_function()
        return child1

    # average crossover function
    def avg_crossover(self, father, mother):

        child1l = []
        child1 = Gene(self.degree, self.points, self.low, self.high)

        # put first part of 1 parent to each child
        child1l = []

        # put the rest of other parent
        for i in range(self.degree):
            child1l[i] = (father.gene[i] + mother.gene[i]) / 2

        # initialize childs gene object
        child1.gene = child1l
        child1.fitness = child1.fitness_function()
        
        return child1


    # random bit crossover function
    def rnd_crossover(self, father, mother):

        child1l = []
        child1 = Gene(self.degree, self.points, self.low, self.high)

        # specify the prob of selecting each parent's gene based on their fitness
        if father.fitness > mother.fitness:
            p_select = 0.3
        else:
            p_select = 0.7

        # put mother or father's gene in child gene with prob p_select
        for i in range(self.degree):
            e_select = random.random()
            if e_select > p_select:
                child1l[i] = father.gene[i]
            else:
                child1l[i] = mother.gene[i]


        # initialize childs gene object
        child1.gene = child1l
        child1.fitness = child1.fitness_function()
        
        return child1



In [280]:
def genetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_replace, pop_size, tornoment_N):
    best_cost = math.inf
    best_result = []
    # load test case
    curve_test1 = Data(test_address)
    curve_test1.load_data()
    print(curve_test1.degree)

    # create first population
    population = Population(curve_test1, pop_size=pop_size)
    population.initialize_population()

    for g_num in range(generation_num):
        # crossover and create as many children as parents population
        children_population = Population(curve_test1, pop_size=pop_size)
        counter = 0
        while counter < pop_size:
            # select 2 parents for crossover
            # father1, mother1 = population.parent_selection(N=tornoment_N)
            # father2, mother2 = population.parent_selection(N=tornoment_N)

            father1, mother1 = population.parent_selection_rank_based()
            father2, mother2 = population.parent_selection_rank_based()

            # crossover 2 selected parents with p_cross
            p_cross = 0.75
            e_cross = random.random()
            if e_cross < p_cross:
                child1 = population.crossover(father1, mother1)
                child2  = population.crossover(father2, mother2)

                # child1 = population.blx_alpha_crossover(father1, mother1, alpha=0.6)
                # child2 = population.blx_alpha_crossover(father2, mother2, alpha=0.65)

                # child1 = population.avg_crossover(father1, mother1)
                # child2 = population.avg_crossover(father2, mother2)

                # child1 = population.rnd_crossover(father1, mother1)
                # child2 = population.rnd_crossover(father2, mother2)
                counter += 2

                # mutate childs with p_mutate
                p_mutate = 0.5
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                children_population.population.append(child1)
                children_population.population.append(child2)
        
        # population replacement
        parent_values = []
        child_values = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness_function()
            y = children_population.population[i].fitness_function()
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))  # find the worst parent
            min_child_ind = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # find the best result in population
        for i in range(pop_size):
            if population.population[i].fitness_function() < best_cost:
                best_cost = population.population[i].fitness_function()
                best_result = population.population[i].gene.copy()

        print(f"generation {g_num}, best_cost so far: {best_cost}")

    return best_result, curve_test1

            


In [281]:
best_result, test = genetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test1.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=100, tornoment_N=5)

10
generation 0, best_cost so far: 5342830246.093042
generation 1, best_cost so far: 3456236792.1555314
generation 2, best_cost so far: 2179678136.567788
generation 3, best_cost so far: 2179678136.567788
generation 4, best_cost so far: 2179678136.567788
generation 5, best_cost so far: 2179678136.567788
generation 6, best_cost so far: 2179678136.567788
generation 7, best_cost so far: 2179678136.567788
generation 8, best_cost so far: 2179678136.567788
generation 9, best_cost so far: 2179678136.567788
generation 10, best_cost so far: 645851589.8361272
generation 11, best_cost so far: 645851589.8361272
generation 12, best_cost so far: 645851589.8361272
generation 13, best_cost so far: 356067861.23851323
generation 14, best_cost so far: 356067861.23851323
generation 15, best_cost so far: 356067861.23851323
generation 16, best_cost so far: 278215919.85407233
generation 17, best_cost so far: 278215919.85407233
generation 18, best_cost so far: 228881390.84906125
generation 19, best_cost so far

KeyboardInterrupt: 

In [ ]:
best_result, test = genetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test1.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=100, tornoment_N=5)

In [277]:
def memetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_neighbor, p_replace, pop_size, tornoment_N):
    best_cost = math.inf
    best_result = []
    # load test case
    curve_test1 = Data(test_address)
    curve_test1.load_data()

    # create first population
    population = Population(curve_test1, pop_size=pop_size)
    population.initialize_population()

    for g_num in range(generation_num):
        # crossover and create as many children as parents population
        children_population = Population(curve_test1, pop_size=pop_size)
        counter = 0
        while counter < pop_size:
            # select 2 parents for crossover
            # father1, mother1 = population.parent_selection(N=tornoment_N)
            # father2, mother2 = population.parent_selection(N=tornoment_N)
            father1, mother1 = population.parent_selection_rank_based()
            father2, mother2 = population.parent_selection_rank_based()

            # crossover 2 selected parents with p_cross
            e_cross = random.random()
            if e_cross < p_cross:
                # child1, child2 = population.crossover(father, mother)
                child1 = population.avg_crossover(father1, mother1)
                child2 = population.avg_crossover(father2, mother2)
                # child1 = population.rnd_crossover(father1, mother1)
                # child2 = population.rnd_crossover(father2, mother2)
                counter += 2

                # mutate childs with p_mutate
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                # do local search to replace best neighbor with p_neighbor
                e_neighbor1 = random.random()
                if e_neighbor1 < p_neighbor:
                    genei = child1.local_search()
                else:
                    genei = child1
                e_neighbor2 = random.random()
                if e_neighbor2 < p_neighbor:
                    genej = child2.local_search()
                else:
                    genej = child2

                

                children_population.population.append(genei)
                children_population.population.append(genej)
        
        # population replacement
        parent_values = []
        child_values = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness_function()
            y = children_population.population[i].fitness_function()
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))  # find the worst parent
            min_child_ind = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # find the best result in population
        for i in range(pop_size):
            if population.population[i].fitness_function() < best_cost:
                best_cost = population.population[i].fitness_function()
                best_result = population.population[i].gene.copy()

        print(f"generation {g_num}, best_cost so far: {best_cost}")

    return best_result, curve_test1

            


In [278]:
best_result, test = memetic_algorithm_main(test_address="CurveFitting-Tests/CurveFitting_test1.txt", generation_num=500, p_cross=0.65, p_mutate=0.5, p_neighbor=0.5,p_replace=0.1, pop_size=100, tornoment_N=5)

generation 0, best_cost so far: 4231672394.2360086
generation 1, best_cost so far: 120102564.64940152
generation 2, best_cost so far: 120102564.64940152
generation 3, best_cost so far: 120102564.64940152
generation 4, best_cost so far: 120102564.64940152
generation 5, best_cost so far: 120102564.64940152
generation 6, best_cost so far: 120102564.64940152
generation 7, best_cost so far: 120102564.64940152
generation 8, best_cost so far: 120102564.64940152
generation 9, best_cost so far: 6295197.574717359
generation 10, best_cost so far: 6295197.574717359
generation 11, best_cost so far: 6295197.574717359
generation 12, best_cost so far: 6295197.574717359
generation 13, best_cost so far: 6295197.574717359
generation 14, best_cost so far: 6295197.574717359
generation 15, best_cost so far: 6295197.574717359
generation 16, best_cost so far: 6295197.574717359
generation 17, best_cost so far: 6295197.574717359
generation 18, best_cost so far: 6105630.064677676
generation 19, best_cost so far:

KeyboardInterrupt: 

In [179]:
import matplotlib.pyplot as plt

# print(np.polyval(best_result, test.points[0][0]))
# print(test.points[0][1])
# Sample data for demonstration
x_values = [x[0] for x in test.points] # Replace with your actual x values
real_y_values = [x[1] for x in test.points]  # Replace with your actual real y values
list1, list2 = zip(*sorted(zip(x_values, real_y_values)))
# Convert back 
x_values = list(list1)
real_y_values = list(list2)
predicted_y_values = [np.polyval(best_result, x[0]) for x in test.points] # Replace with your actual predicted y values
predicted_y_values = sorted(predicted_y_values)
print(real_y_values)
print(predicted_y_values)
# Plot the real curve
# plt.plot(x_values, real_y_values, label='Real Curve', color='blue')

# # Plot the predicted curve
# plt.plot(x_values, predicted_y_values, label='Predicted Curve', color='red', linestyle='dashed')

# # Add labels and legend
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.title('Real vs Predicted Curve')
# plt.legend()

# # Show the plot
# plt.show()


[-379949829948.33, -293824077866.055, -188223396928.236, -185473021949.755, -177924482448.454, -136330383382.666, -100279320986.428, -52776103125.97, -49738997442.699, -46608871671.366, -45674180389.985, -26843813714.295, -22053636969.64, -20492841064.698, -17292911708.105, -12589732555.14, -8328543886.293, -69163956.292, -6957597.593, -6667993.112, -2819043.785, -1298329.036, -278255.65, -109767.257, -10454.45, -7646.216, -469.248, -0.029, 0.012, 0.116, 1271.025, 856522.613, 1138355.064, 62406775.971, 122300904.007, 270930418.949, 1432475288.482, 1439455117.674, 2518851793.41, 3126351723.071, 3180722080.763, 4620594285.391, 6746495432.931, 10292921305.449, 24513263630.634, 44237501555.532, 60662510535.694, 181076919229.972, 199723057156.746, 381827746452.189]
[-379659182685.9307, -293671782898.536, -188223855601.3563, -185422293601.90765, -177952866291.17657, -136357084278.9151, -100314553610.8545, -52847673423.58088, -49798344582.44669, -46660990868.672485, -45744004629.474915, -2688

In [104]:
# load test case
curve_test1 = Data("CurveFitting-Tests/CurveFitting_test1.txt")
curve_test1.load_data()

# create first population
population = Population(curve_test1, pop_size=100)
population.initialize_population()

# select 2 parents for crossover
father, mother = population.parent_selection(N=4)

# crossover 2 selected parents with p_cross
p_cross = 0.75
e_cross = random.random()
if e_cross < p_cross:
    child1, child2 = population.crossover(father, mother)

# mutate childs with p_mutate
p_mutate = 0.5
e_mutate1 = random.random()
if e_mutate1 < p_mutate:
    child1.mutation()
e_mutate2 = random.random()
if e_mutate2 < p_mutate:
    child2.mutation()

# population replacement
parent_values = []
child_values = []
for i in range(population.population_size):             # save all values of parents and children in 2 value lists
    x = population.population[i].fitness_function()
    y = evaluation_function(children_population[i], stock_len)
    parent_values.append(x)
    child_values.append(y)

num_replace = replacement_rate * len(population)    # find the number of parents that should be replaced with children in population

for i in range(int(num_replace)):        # replace as the size of replacements in population
    max_parent_ind = parent_values.index(max(parent_values))  # find the worst parent
    min_child_ind = child_values.index(min(child_values))      # find best children

    population.pop(max_parent_ind)      # remove worst parent from population
    population.append(children_population[min_child_ind])  # add best children to population

    children_population.pop(min_child_ind)  # remove best children from population of children
    parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
    child_values.pop(min_child_ind)  # remove best children value from children values

# find the best result in population
for i in range(len(population)):
if evaluation_function(population[i], stock_len) < best_cost:
    best_cost = evaluation_function(population[i], stock_len)
    best_result = population[i]
